In [2]:
using PerlaTonettiWaugh

## Overview

This notebook reads in the calibrated parameter values and then computes the results associated with the transition dynamics appearing in Section 7-4. It is stripped down and just focuses on results in the paper. More analysis of the output/reults are presented in the notebook [TransitionDynamics_Full.ipynb](TransitionDynamics_Full.ipynb). 

Key components of the notebook are:

- [Steady State Analysis](#steady)

- [Transition Dynamics Computation](#trans)

- [Welfare](#welfare)

---
### <a name="steady"></a>Steady State Analysis

In [9]:
params = load_parameters("parameters/calibration_params.csv")
baseline = merge(params, (d = params.d_0,))
baseline_sol = stationary_algebraic(baseline, settings_defaults());

In [10]:
new_params = merge(params, (d = params.d_T,))
new_sol = stationary_algebraic(new_params, settings_defaults());

In [13]:
println("")
println("-------------------------------------------------------------------------")
println("SS to SS Baseline vs. 10% Lower Trade Costs")
println("")


-------------------------------------------------------------------------
SS to SS Baseline vs. 10% Lower Trade Costs



In [14]:
println("")
println("Change in Growth, Steady State to Steady State")
println("Growth in Baseline: ", round(100*baseline_sol.g, digits=2))
println("Growth in New SS: ", round(100*new_sol.g, digits=2))
println("")


Change in Growth, Steady State to Steady State
Growth in Baseline: 0.79
Growth in New SS: 1.03



In [15]:
println("")
println("Imports/GDP, Steady State to Steady State")
println("Imports/GDP in Baseline: ", round(100*(1 - baseline_sol.λ_ii), digits=1))
println("Imports/GDP in New SS: ", round(100*(1 - new_sol.λ_ii), digits=1))
println("")


Imports/GDP, Steady State to Steady State
Imports/GDP in Baseline: 10.6
Imports/GDP in New SS: 14.4



In [16]:
ss_ce = 100*consumption_equivalent(new_sol.U_bar, baseline_sol.U_bar, params);
println("")
println("Consumption Equivalent Welfare Gain, Steady State to Steady State")
println("CE gains: ", round(ss_ce, digits=1))
println("")
println("")
println("-------------------------------------------------------------------------")


Consumption Equivalent Welfare Gain, Steady State to Steady State
CE gains: 11.2



---

### <a name="trans"></a>Transition Dynamics Computation

In [17]:
using CSV, Plots, JSON, BenchmarkTools, Parameters, DataFrames, Suppressor
gr(fmt = :png)

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1260
┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1260


Plots.GRBackend()

In [18]:
overall_parameters = load_parameters("parameters/calibration_params.csv")

(ρ = 0.0203380446685169, σ = 3.16692413583811, θ = 4.98897658793826, κ = 0.104196324793307, χ = 0.126846612050694, μ = -0.0310646242175711, υ = 0.0483011406016648, ζ = 1, δ = 0.02, N = 10, γ = 1, η = 0, Theta = 1, d_0 = 3.0224928254626, d_T = 2.82024354291634)

In [19]:
settings = settings_defaults(); # numerical solution settings
@unpack stationary_T, stationary_0, change_welfare, change_trade = compare_steady_states(overall_parameters, settings);

Note that everything from here on out is using the **numerical** solution, so the values might be slightly different than the algebraic one. 

We do this so that the endpoints are valid targets for the dynamical experiment. 

To use the algebraic one, simply add `algebraic = true` to the `compare_steady_states` function above. 

In [20]:
#@show (stationary_T.g, stationary_T.z_hat, stationary_T.Ω, stationary_T.L_tilde); # inspect time T solution

(stationary_T.g, stationary_T.z_hat, stationary_T.Ω, stationary_T.L_tilde) = (0.01025616153876511, 1.8523062408850237, 0.6508139010465566, 0.24606267582511263)


In [21]:
#@show (stationary_0.g, stationary_0.z_hat, stationary_0.Ω, stationary_0.L_tilde); # inspect time 0 solution

(stationary_0.g, stationary_0.z_hat, stationary_0.Ω, stationary_0.L_tilde) = (0.007920170474460874, 1.9881975410332189, 0.6852422606329861, 0.24226457898683534)


In [22]:
cachename = model_cachename(overall_parameters, settings) # uniquely identify the run by parameters + settings
cachepath = joinpath(pwd(), "data", cachename * ".csv")
use_cache = true # change this to false if you don't want to use the cache for a run
write_cache = true # change this to false if you don't want to generate *any* cache files for a run

true

In [29]:
println("")
println("Computing Transition Dynamics")


Computing Transition Dynamics


In [23]:
cachename # this is the hash for this run
println("")
println("cachename: ", cachename)


"14593292395764940198"

In [24]:
if isfile(cachepath) && use_cache # read if possible
    println("")
    println("using cache.")
    solved = CSV.read(cachepath);
else
    println("")
    println("solving without cache")
    @time solution = @suppress solve_transition(overall_parameters, settings);
    solved = solution.results 
end

solving without cache.
491.598072 seconds (1.80 G allocations: 693.922 GiB, 14.64% gc time)


,t,g,z_hat,Ω,E,v_1,L_tilde,entry_residual
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-20.0,0.00792017,1.9882,0.685242,0.02,6.86678,0.242265,-2.42473e-13
2,-15.0,0.00792017,1.9882,0.685242,0.02,6.86678,0.242265,-2.42473e-13
3,-10.0,0.00792017,1.9882,0.685242,0.02,6.86678,0.242265,-2.42473e-13
4,-5.0,0.00792017,1.9882,0.685242,0.02,6.86678,0.242265,-2.42473e-13
5,-1.0,0.00792017,1.9882,0.685242,0.02,6.86678,0.242265,-2.42473e-13
6,0.0,0.00898344,1.87504,0.685242,0.0158639,6.86678,0.230633,0.0
7,0.25,0.00899195,1.87488,0.68493,0.0159106,6.86678,0.230821,0.0
8,0.5,0.00900046,1.87473,0.684619,0.0159571,6.86678,0.231007,0.0
9,0.75,0.00900895,1.87457,0.684309,0.0160032,6.86678,0.231192,0.0


In [25]:
if write_cache
    mkpath(joinpath(pwd(), "data")); # if this doesn't eist for whatever reason
    CSV.write(cachepath, solved); # write DataFrame
    write(joinpath(pwd(), "data", cachename * ".json"), JSON.json((parameters = overall_parameters, settings = merge(settings, (interp = typeof(settings.interp), stationary_x0 = typeof(settings.stationary_x0), fixedpoint_x0 = typeof(settings.fixedpoint_x0) ))))); # write parameters
else
    # nothing
end

6323

---

### <a name="welfare"></a> Welfare

Here's welfare inclusive of the transition path:

In [28]:
ce_transition = 100*consumption_equivalent(solved.U[6], stationary_0.U_bar, load_parameters("parameters/calibration_params.csv"));
println("")
println("-------------------------------------------------------------------------")
println("")
println("Consumption Equivalent Welfare Gain, Inclusive of Transition Path")
println("CE gains: ", round(ce_transition, digits=1))
println("")


Consumption Equivalent Welfare Gain, Steady State to Steady State
CE gains: 10.8

